# Features Engineering

In this notebook we experiment with some features engineering:

* using **TSFresh**

* and using **ROCKET** (Random Convolutional Kernel Transform)


**NOTE:** Before starting exploring this notebook, I recommend checking `1-EDA.ipynb` notebook first - it contains Exploratory Data Analysis and will help you get some understanding of the datasets. 

---

In [1]:
import warnings
warnings.filterwarnings('ignore')

from datetime import datetime

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns

%matplotlib inline

First, let's read aircraft engines datasets, we start with "FD001" dataset, which has:

* 100 engines time series in TRAIN set

* 100 engines time series in TEST set

* 1 Fault condition

* 1 Operating condition


In [2]:
from utils import read_dataset, calculate_RUL, SENSOR_COLUMNS

train, test, test_rul = read_dataset('FD001')

train['rul'] = calculate_RUL(train, upper_threshold=135)

print(f'train.shape = {train.shape}')
train.head(2)

train.shape = (20631, 27)


,unit,time_cycles,op_setting_1,op_setting_2,op_setting_3,sensor_1,sensor_2,sensor_3,sensor_4,sensor_5,...,sensor_13,sensor_14,sensor_15,sensor_16,sensor_17,sensor_18,sensor_19,sensor_20,sensor_21,rul
0,1,1,-0.0007,-0.0004,100.0,518.67,641.82,1589.70,1400.60,14.62,...,2388.02,8138.62,8.4195,0.03,392,2388,100.0,39.06,23.4190,135
1,1,2,0.0019,-0.0003,100.0,518.67,642.15,1591.82,1403.14,14.62,...,2388.07,8131.49,8.4318,0.03,392,2388,100.0,39.00,23.4236,135


Before starting features engineering - we should remove sensors with constant values that we saw during Exploratory Data Analysis. For that, we create a transformer which drops features with variance lower than a `threshold` - see below:

In [3]:
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import VarianceThreshold


class LowVarianceFeaturesRemover(BaseEstimator, TransformerMixin):
    def __init__(self, threshold=0):
        self.threshold = threshold
        self.selector = VarianceThreshold(threshold=threshold)

    def fit(self, X):
        self.selector.fit(X)
        return self

    def transform(self, X):
        X_t = self.selector.transform(X)
        droped_features = X.columns[~self.selector.get_support()]
        print(f'Droped low variance features: {droped_features.to_list()}')
        return pd.DataFrame(X_t, columns=self.selector.get_feature_names_out())

Scaling per engines from `1-EDA.ipynb` notebook:

In [4]:
from utils import SENSOR_COLUMNS


class ScalePerEngine(BaseEstimator, TransformerMixin):
    '''
    Scale individual engines time series with respect to its start.
    Substract firts `n_first_cycles` AVG values from time series.
    '''
    def __init__(self, n_first_cycles=20, sensors_columns=SENSOR_COLUMNS):
        self.n_first_cycles = n_first_cycles
        self.sensors_columns = sensors_columns

    def fit(self, X):
        return self

    def transform(self, X):
        self.sensors_columns = [x for x in X.columns if x in self.sensors_columns]

        init_sensors_avg = X[X['time_cycles'] <= self.n_first_cycles] \
            .groupby(by=['unit'])[self.sensors_columns] \
            .mean() \
            .reset_index()

        X_t = X[X['time_cycles'] > self.n_first_cycles].merge(
            init_sensors_avg,
            on=['unit'], how='left', suffixes=('', '_init_v')
        )

        for SENSOR in self.sensors_columns:
            X_t[SENSOR] = X_t[SENSOR] - X_t['{}_init_v'.format(SENSOR)]

        drop_columns = X_t.columns.str.endswith('init_v')
        return X_t[X_t.columns[~drop_columns]]


---

# Features Engineering with TSfresh

**TSFresh** (https://github.com/blue-yonder/tsfresh) extracts a large number of features from time series and has a built-in features filtering procedure.

You can check a list of features TSfresh extracts here: 
* https://tsfresh.readthedocs.io/en/latest/api/tsfresh.feature_extraction.html#module-tsfresh.feature_extraction.feature_calculators.



**NOTE:**

While TSfresh automates a lot of features engineering - it also **tends to extract highly correlated features** - see [1]. In order to eliminate this problem, the authors of the paper and the library propose to perform Principal Component Analysis. PCA can be done right after TSfresh generates a candidate features and before the features selection procedure. Or PCA can be performed after TSFresh does features selection. We stick to the first option and apply features selection on principal components.


[1] Christ, M., Kempa-Liehr, A.W. and Feindt, M. (2016). Distributed and parallel time series feature extraction for industrial big data applications. ArXiv e-prints: 1610.07717 URL: http://adsabs.harvard.edu/abs/2016arXiv161007717C



<br>
Our tsfresh features engineering pipeline is:

1. transform the original time series into sliding windows of length 30 (see `roll_time_series` method below)

2. calculate TSFresh features from the predefined list of features (see `tsfresh_calc` below)

3. perform PCA

4. do features selection with TSfresh 

In [5]:
from tsfresh.utilities.dataframe_functions import roll_time_series


class RollTimeSeries(BaseEstimator, TransformerMixin):
    def __init__(self, min_timeshift, max_timeshift, rolling_direction):
        self.min_timeshift = min_timeshift
        self.max_timeshift = max_timeshift
        self.rolling_direction = rolling_direction
        
    def fit(self, X):
        return self
    
    def transform(self, X):
        _start = datetime.now()
        print('Start Rolling TS')
        X_t = roll_time_series(
                    X, column_id='unit', column_sort='time_cycles', 
                    rolling_direction=self.rolling_direction,
                    min_timeshift=self.min_timeshift,
                    max_timeshift=self.max_timeshift)
        print(f'Done Rolling TS in {datetime.now() - _start}')
        return X_t

<br>

**NOTE:** TSfresh suggests a couple of options to choose a set of features from - e.g. `MinimalFCParameters`, `ComprehensiveFCParameters`, `EfficientFCParameters` - see https://tsfresh.readthedocs.io/en/latest/text/feature_extraction_settings.html.

However, after reviewing the proposed features lists, a set of specific features was shortlisted - see below: 

In [6]:
tsfresh_calc = {
    'mean_change': None,
    'mean': None,
    'standard_deviation': None,
    'root_mean_square': None,
    'last_location_of_maximum': None,
    'first_location_of_maximum': None,
    'last_location_of_minimum': None,
    'first_location_of_minimum': None,
    'maximum': None,
    'minimum': None,
    'time_reversal_asymmetry_statistic': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
    'c3': [{'lag': 1}, {'lag': 2}, {'lag': 3}],
    'cid_ce': [{'normalize': True}, {'normalize': False}],
    'autocorrelation': [{'lag': 0},
        {'lag': 1},
        {'lag': 2},
        {'lag': 3},],
    'partial_autocorrelation': [{'lag': 0},
        {'lag': 1},
        {'lag': 2},
        {'lag': 3},],
    'linear_trend': [
        {'attr': 'intercept'},
        {'attr': 'slope'},
        {'attr': 'stderr'}],
    'augmented_dickey_fuller': [{'attr': 'teststat'},
        {'attr': 'pvalue'},
        {'attr': 'usedlag'}],
    'linear_trend_timewise': [
        {'attr': 'intercept'},
        {'attr': 'slope'}],
    'lempel_ziv_complexity': [{'bins': 2},
        {'bins': 3},
        {'bins': 5},
        {'bins': 10},
        {'bins': 100}],
    'permutation_entropy': [{'tau': 1, 'dimension': 3},
        {'tau': 1, 'dimension': 4},
        {'tau': 1, 'dimension': 5},
        {'tau': 1, 'dimension': 6},
        {'tau': 1, 'dimension': 7}],
    'fft_coefficient': [
        {'coeff': 0, 'attr': 'abs'},
        {'coeff': 1, 'attr': 'abs'},
        {'coeff': 2, 'attr': 'abs'},
        {'coeff': 3, 'attr': 'abs'},
        {'coeff': 4, 'attr': 'abs'},
        {'coeff': 5, 'attr': 'abs'},
        {'coeff': 6, 'attr': 'abs'},
        {'coeff': 7, 'attr': 'abs'},
        {'coeff': 8, 'attr': 'abs'},
        {'coeff': 9, 'attr': 'abs'},
        {'coeff': 10, 'attr': 'abs'},],
    'fft_aggregated': [{'aggtype': 'centroid'},
        {'aggtype': 'variance'},
        {'aggtype': 'skew'},
        {'aggtype': 'kurtosis'}],
}

In [7]:
from tsfresh import extract_features, select_features
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA

from utils import calculate_RUL, SENSOR_COLUMNS

In [8]:
class TSFreshFeaturesExtractor(BaseEstimator, TransformerMixin):
    def __init__(self, calc=tsfresh_calc):
        self.calc = calc
        
    def _clean_features(self, X):
        old_shape = X.shape
        X_t = X.T.drop_duplicates().T
        print(f'Droped {old_shape[1] - X_t.shape[1]} duplicate features')

        old_shape = X_t.shape
        X_t = X_t.dropna(axis=1)
        print(f'Droped {old_shape[1] - X_t.shape[1]} features with NA values')
        return X_t
    
    def fit(self, X):
        return self
    
    def transform(self, X):
        _start = datetime.now()
        print('Start Extracting Features')
        X_t = extract_features(
            X[['id', 'time_cycles'] 
               + X.columns[X.columns.str.startswith('sensor')].tolist()], 
            column_id='id', 
            column_sort='time_cycles', 
            default_fc_parameters=self.calc)
        print(f'Done Extracting Features in {datetime.now() - _start}')
        X_t = self._clean_features(X_t)
        return X_t

    
class CustomPCA(BaseEstimator, TransformerMixin):
    def __init__(self, n_components=None, random_state=None):
        self.n_components = n_components
        self.random_state = random_state
        
    def fit(self, X):
        assert 'unit' not in X.columns, "columns should be only features"
        self.ftr_columns = X.columns

        self.scaler = StandardScaler()
        X_sc = self.scaler.fit_transform(X[self.ftr_columns].values)

        self.pca = PCA(n_components=self.n_components, random_state=self.random_state)
        self.pca.fit_transform(X_sc)
        return self

    def transform(self, X):
        X_sc = self.scaler.transform(X[self.ftr_columns].values)
        X_pca = self.pca.transform(X_sc)        
        return pd.DataFrame(X_pca, index=X.index)

    

class TSFreshFeaturesSelector(BaseEstimator, TransformerMixin):
    def __init__(self, fdr_level=0.001):
        self.fdr_level = fdr_level

    def fit(self, X):
        rul = calculate_RUL(
            X.index.to_frame(name=['unit', 'time_cycles']).reset_index(drop=True),
            upper_threshold=135)

        X_t = select_features(
            X, rul, fdr_level=self.fdr_level
        )
        self.selected_ftr = X_t.columns
        
        print(f'Selected {len(self.selected_ftr)} out of {X.shape[1]} features: '
              f'{self.selected_ftr.to_list()}')
        return self

    def transform(self, X):
        return X[self.selected_ftr]


In [9]:
from sklearn.pipeline import Pipeline

In [10]:
tsfresh_pipe = Pipeline([
    # Cleaning
    ('drop-low-variance', LowVarianceFeaturesRemover(threshold=0)),
    
    # Scaling and Preprocessing
    ('scale-per-engine', ScalePerEngine(n_first_cycles=15, sensors_columns=SENSOR_COLUMNS)),
    ('roll-time-series', RollTimeSeries(min_timeshift=29, max_timeshift=29, rolling_direction=1)),
    
    # TSFresh features engineering
    ('extract-tsfresh-features', TSFreshFeaturesExtractor(calc=tsfresh_calc)),
    ('PCA', CustomPCA(n_components=40)),
    ('features-selection', TSFreshFeaturesSelector(fdr_level=0.001)),
])

In [11]:
train_tsfresh_ftr = tsfresh_pipe.fit_transform(train)

train_tsfresh_ftr.head(2)

Droped low variance features: ['op_setting_3', 'sensor_1', 'sensor_5', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19']
Start Rolling TS


Rolling: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:10<00:00,  1.90it/s]


Done Rolling TS in 0:00:11.210756
Start Extracting Features


Feature Extraction: 100%|████████████████████████████████████████████████████████| 20/20 [11:57<00:00, 35.86s/it]


Done Extracting Features in 0:12:29.266573
Droped 19 duplicate features
Droped 11 features with NA values
Selected 9 out of 40 features: [0, 2, 3, 4, 5, 1, 10, 31, 19]


0         2         3         4         5         1   \
1.0 45.0 -10.987398 -0.982110  6.989796 -2.297053 -0.054998  0.065238   
    46.0 -10.781827 -1.067625  6.895214 -2.323534 -0.224924  0.476905   

                10        31        19  
1.0 45.0  1.771415  1.013616 -1.271091  
    46.0  1.526736  1.240907 -1.646843

---

# Features Engineering with ROCKET

ROCKET (Random Convolutional Kernel Transform) transforms time series into features using random convolutional kernels (by default 10000 kernels) and applying max pooling and proportion of positive values (which produces 2 features per kernel). This results in (by default) 20000 features.  

[1] Dempster A., Petitjean F. and Webb G., "ROCKET: Exceptionally fast and accurate time series
classification using random convolutional kernels": https://arxiv.org/pdf/1910.13051.pdf

Note: here we add extra step for scaling the time series before running ROCKET - see below:

In [12]:
from sklearn.preprocessing import StandardScaler


class CustomStandardScaler(BaseEstimator, TransformerMixin):
    def fit(self, X):
        self.scale_columns = X.columns[X.columns.str.startswith('sensor')
                                       | X.columns.str.startswith('op_setting')]
        self.scaler = StandardScaler()
        self.scaler.fit(X[self.scale_columns])
        return self
    
    def transform(self, X):
        X_t = self.scaler.transform(X[self.scale_columns])
        df = pd.concat([
                X[[c for c in X.columns if c not in self.scale_columns]],
                pd.DataFrame(X_t, columns=self.scale_columns)
            ], axis=1).sort_values(['unit', 'time_cycles']).reset_index(drop=True)
        return df


In [13]:
from datetime import datetime

from sktime.datatypes._panel._convert import from_multi_index_to_nested


class TransformTS2Nested(BaseEstimator, TransformerMixin):
    '''
    See https://www.sktime.org/en/stable/examples/loading_data.html#Using-multi-indexed-pandas-DataFrames
    '''
    def fit(self, X):
        return self
    
    def transform(self, X):
        assert 'id' in X.columns, "X should have `id` column"
        X_mi = X.copy()
        X_mi.index = pd.MultiIndex.from_frame(X[['id', 'time_cycles']])
        X_mi = X_mi.drop(columns=['rul', 'id'], errors='ignore')
        
        _start = datetime.now()
        print('Start Converting multi-index DF to sktime nested DF')
        X_nested = from_multi_index_to_nested(X_mi, instance_index='id')
        
        X_nested.index = pd.MultiIndex.from_frame(
            pd.DataFrame({'unit': X_nested['unit'].apply(lambda x: x.unique()[0]),
                          'time_cycles': X_nested['time_cycles'].apply(lambda x: x.iloc[-1])})
        )
        X_nested = X_nested.drop(columns=['unit', 'time_cycles'])
        print(f'Converted multi-index DF to sktime nested DF in {datetime.now() - _start}')

        return X_nested


In [14]:
from sktime.transformations.panel.rocket import Rocket


class RocketTransform(BaseEstimator, TransformerMixin):
    '''
    See https://www.sktime.org/en/stable/examples/rocket.html
    '''
    def __init__(self, num_kernels=10000, normalise=False, n_jobs=-1, random_state=None):
        self.num_kernels = num_kernels
        self.normalise = normalise
        self.n_jobs = n_jobs
        self.random_state = random_state

    def fit(self, X):
        self.rocket = Rocket(
            num_kernels=self.num_kernels, 
            normalise=self.normalise, 
            n_jobs=self.n_jobs,
            random_state=self.random_state)
        self.rocket.fit(X)
        return self
    
    def transform(self, X):
        rocket_t = self.rocket.transform(X)
        rocket_t.index = X.index
        return rocket_t


In [15]:
from sklearn.pipeline import Pipeline

from utils import SENSOR_COLUMNS


rocket_pipe = Pipeline([
    # Cleaning constant features
    ('drop-low-variance', LowVarianceFeaturesRemover()),
    
    # Scaling sensors values
    ('scale-per-engine', ScalePerEngine(n_first_cycles=15, sensors_columns=SENSOR_COLUMNS)),
    ('scale', CustomStandardScaler()),
    
    # Preprocessing for ROCKET
    ('roll-time-series', RollTimeSeries(min_timeshift=29, max_timeshift=29, rolling_direction=1)),
    ('nest-time-series', TransformTS2Nested()),
    
    # Features Engineering w ROCKET
    ('rocket', RocketTransform(num_kernels=10000, normalise=False, random_state=2021))
])



In [16]:
train_rocket_ftr = rocket_pipe.fit_transform(train)

train_rocket_ftr.shape

Droped low variance features: ['op_setting_3', 'sensor_1', 'sensor_5', 'sensor_10', 'sensor_16', 'sensor_18', 'sensor_19']
Start Rolling TS


Rolling: 100%|███████████████████████████████████████████████████████████████████| 20/20 [00:08<00:00,  2.27it/s]


Done Rolling TS in 0:00:09.353206
Start Converting multi-index DF to sktime nested DF
Converted multi-index DF to sktime nested DF in 0:07:41.130546


(16231, 20000)

Note that here we end up with `p >> n` case.

---